# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the XES file
    dataframe_log = pm4py.read_xes('../../data/logs/DomesticDeclarations.xes')

    # If 'log' is already a DataFrame, add the @@case_index column directly
    case_indices = {case_id: idx for idx, case_id in enumerate(dataframe_log['case:concept:name'].unique())}
    dataframe_log['@@case_index'] = dataframe_log['case:concept:name'].map(case_indices)
    
     # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)
    
dataframe_log

/opt/anaconda3/lib/python3.9/site-packages/pm4py/util/dt_parsing/parser.py:76: UserWarning: ISO8601 strings are not fully supported with strpfromiso for Python versions below 3.11
  warnings.warn(


parsing log, completed traces ::   0%|          | 0/10500 [00:00<?, ?it/s]

,id,org:resource,concept:name,time:timestamp,org:role,case:id,case:concept:name,case:BudgetNumber,case:DeclarationNumber,case:Amount,@@case_index
0,st_step 86794_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 08:49:50+00:00,EMPLOYEE,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
1,st_step 86793_0,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2017-01-09 10:27:48+00:00,SUPERVISOR,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
2,dd_declaration 86791_19,SYSTEM,Request Payment,2017-01-10 08:34:44+00:00,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
3,dd_declaration 86791_20,SYSTEM,Payment Handled,2017-01-12 16:31:22+00:00,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
4,st_step 86798_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2017-01-09 09:26:14+00:00,EMPLOYEE,declaration 86795,declaration 86795,budget 86566,declaration number 86796,182.464172,1
...,...,...,...,...,...,...,...,...,...,...,...
56432,st_step 138363_0,STAFF MEMBER,Declaration SUBMITTED by EMPLOYEE,2018-12-29 16:50:14+00:00,EMPLOYEE,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499
56433,st_step 138361_0,STAFF MEMBER,Declaration APPROVED by ADMINISTRATION,2018-12-29 16:56:13+00:00,ADMINISTRATION,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499
56434,st_step 138362_0,STAFF MEMBER,Declaration FINAL_APPROVED by SUPERVISOR,2019-01-03 07:55:52+00:00,SUPERVISOR,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499
56435,dd_declaration 138359_19,SYSTEM,Request Payment,2019-01-08 07:20:28+00:00,UNDEFINED,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499


# Insert Start & End markers

In [2]:
# Define a function to insert start and end markers
def add_markers(df):
    # Identify unique case indices
    case_indices = df['@@case_index'].unique()
    
    # Prepare a container for new DataFrame rows
    new_rows = []
    
    # Iterate over each case index to add start and end markers
    for case_index in case_indices:
        # Create a start marker row with all columns except @@case_index set to 'Start'
        start_row = {col: 'Start' if col != '@@case_index' else case_index for col in df.columns}
        
        # Create an end marker row with all columns except @@case_index set to 'End'
        end_row = {col: 'End' if col != '@@case_index' else case_index for col in df.columns}
        
        # Append start row, rows for the current case, and end row
        new_rows.append(start_row)
        new_rows.extend(df[df['@@case_index'] == case_index].to_dict('records'))
        new_rows.append(end_row)
    
    # Convert the list of rows into a DataFrame
    return pd.DataFrame(new_rows)

# Apply the function to add start and end markers to the dataframe
modified_dataframe = add_markers(dataframe_log)

# Preprocess

In [3]:
codes, uniques = pd.factorize(modified_dataframe['concept:name'])
modified_dataframe['concept:name'] = codes

### Padding for Cases with less then 5 events

In [4]:
# Calculating the frequency of each unique value in '@@case_index'
frequency = modified_dataframe['@@case_index'].value_counts()

# Finding the minimum occurrence
min_occurrence = frequency.min()

min_occurrence

3

In [5]:
# Assuming your dataframe might have different types, let's create a generic function to add rows
def add_rows(group, num_rows, case_index_value):
    # For each column, determine the appropriate "zero" value (int 0, string '', etc.)
    additional_rows = pd.DataFrame({
        column: 0 if pd.api.types.is_numeric_dtype(group[column]) else '' for column in group.columns
    }, index=range(num_rows))
    
    # Set the @@case_index column to the current case index value
    additional_rows['@@case_index'] = case_index_value
    
    # Append the additional rows to the group
    return pd.concat([group, additional_rows], ignore_index=True)

# Function to pad cases with less than 5 events
def pad_cases(df):
    # Group by @@case_index
    groups = df.groupby('@@case_index')
    
    # Placeholder for modified groups
    modified_groups = []
    
    for name, group in groups:
        # Calculate the number of events to add
        events_to_add = 5 - len(group)
        
        if events_to_add > 0:
            # Add the required number of rows
            group = add_rows(group, events_to_add, name)
        
        # Append the modified group to the list
        modified_groups.append(group)
    
    # Concatenate all modified groups back into a single DataFrame
    return pd.concat(modified_groups, ignore_index=True) 

In [6]:
# Apply the padding function
modified_dataframe = pad_cases(modified_dataframe)
modified_dataframe

,id,org:resource,concept:name,time:timestamp,org:role,case:id,case:concept:name,case:BudgetNumber,case:DeclarationNumber,case:Amount,@@case_index
0,Start,Start,0,Start,Start,Start,Start,Start,Start,Start,0
1,st_step 86794_0,STAFF MEMBER,1,2017-01-09 08:49:50+00:00,EMPLOYEE,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
2,st_step 86793_0,STAFF MEMBER,2,2017-01-09 10:27:48+00:00,SUPERVISOR,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
3,dd_declaration 86791_19,SYSTEM,3,2017-01-10 08:34:44+00:00,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
4,dd_declaration 86791_20,SYSTEM,4,2017-01-12 16:31:22+00:00,UNDEFINED,declaration 86791,declaration 86791,budget 86566,declaration number 86792,26.851205,0
...,...,...,...,...,...,...,...,...,...,...,...
77705,st_step 138361_0,STAFF MEMBER,12,2018-12-29 16:56:13+00:00,ADMINISTRATION,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499
77706,st_step 138362_0,STAFF MEMBER,2,2019-01-03 07:55:52+00:00,SUPERVISOR,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499
77707,dd_declaration 138359_19,SYSTEM,3,2019-01-08 07:20:28+00:00,UNDEFINED,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499
77708,dd_declaration 138359_20,SYSTEM,4,2019-01-10 16:31:08+00:00,UNDEFINED,declaration 138359,declaration 138359,budget 86566,declaration number 138360,190.404576,10499


# Generate sliding windows

In [7]:
df_activity = modified_dataframe[['concept:name', '@@case_index']]

In [8]:
def generate_sliding_windows(df, case_id_column='@@case_index', window_size=5):
    windows = []
    targets = []
    case_indices = []

    # Iterate over each unique case
    for case_id in df[case_id_column].unique():
        # Extract the case
        case_data = df[df[case_id_column] == case_id]
        
        # Convert case_data to a NumPy array and drop the case_id_column
        case_data_array = case_data.drop(columns=[case_id_column]).to_numpy()

        # Adjusting the condition to correctly reflect window_size without needing an additional +1
        # Now it correctly considers window_size as including the target event
        if len(case_data_array) >= window_size:
            # Adjust the loop to generate sliding windows of size window_size - 1 for the inputs and use the next event as the target
            for i in range(len(case_data_array) - window_size + 1):
                # window now has window_size - 1 events
                window = case_data_array[i:i + window_size - 1]
                # The target is the event immediately following the window
                target = case_data_array[i + window_size - 1]
                windows.append(window)
                targets.append(target)
                case_indices.append(case_id)  # Store the case_id corresponding to the window

    # Convert lists to numpy arrays for easier handling and to ensure they are two-dimensional
    windows_array = np.array(windows)
    targets_array = np.array(targets)
    case_indices_array = np.array(case_indices)
    
    return windows_array, targets_array, case_indices_array

In [9]:
windows_activity, targets_activity, case_indices = generate_sliding_windows(df_activity)

# LSTM

### Architecture

In [10]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

# Assuming these values as placeholders, replace them with actual counts from your data
num_activities = modified_dataframe['concept:name'].nunique()

embedding_dim_activity = 50
time_steps = 4

# Input layer for activities
input_activity = Input(shape=(time_steps,), name='input_activity')

# Embedding layer for activities
embedding_activity = Embedding(input_dim=num_activities, output_dim=embedding_dim_activity, input_length=time_steps)(input_activity)

# LSTM layer for activities
lstm_activity = LSTM(25, return_sequences=False)(embedding_activity)

# Output layer for predicting the next activity
output_activity = Dense(num_activities, activation='softmax', name='output_activity')(lstm_activity)

# Create and compile the model
model = Model(inputs=input_activity, outputs=output_activity)

model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

2024-08-19 21:43:20.665581: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_activity (InputLayer  [(None, 4)]               0         
 )                                                               
                                                                 
 embedding (Embedding)       (None, 4, 50)             950       
                                                                 
 lstm (LSTM)                 (None, 25)                7600      
                                                                 
 output_activity (Dense)     (None, 19)                494       
                                                                 
Total params: 9044 (35.33 KB)
Trainable params: 9044 (35.33 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Data Splitting

In [11]:
from sklearn.model_selection import train_test_split

# Split the activity data
train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)

# Training

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def cyclic_lr(epoch, lr):
    # Example function that modulates LR within a range for each epoch
    # Customize this function based on your cyclic learning rate policy
    max_lr = 0.01  # Maximum LR
    base_lr = 0.001  # Base LR
    step_size = 10  # Number of epochs for half a cycle
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * np.maximum(0, (1 - x))
    return lr

lr_scheduler = LearningRateScheduler(cyclic_lr)

In [13]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Convert activity targets to categorical
train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities)

# Train the model
history = model.fit(train_activity, 
                    train_targets_activity_cat,
                    epochs=25,
                    batch_size=64,
                    validation_data=(test_activity, test_targets_activity_cat),
                    verbose=1,
                    callbacks=[lr_scheduler])  # Add other callbacks as needed

Epoch 1/25
391/391 [==============================] - 4s 4ms/step - loss: 0.9374 - accuracy: 0.7657 - val_loss: 0.2540 - val_accuracy: 0.9542 - lr: 0.0010
Epoch 2/25
391/391 [==============================] - 1s 3ms/step - loss: 0.1469 - accuracy: 0.9675 - val_loss: 0.1023 - val_accuracy: 0.9713 - lr: 0.0019
Epoch 3/25
391/391 [==============================] - 1s 3ms/step - loss: 0.0887 - accuracy: 0.9718 - val_loss: 0.0841 - val_accuracy: 0.9732 - lr: 0.0028
Epoch 4/25
391/391 [==============================] - 1s 3ms/step - loss: 0.0773 - accuracy: 0.9733 - val_loss: 0.0777 - val_accuracy: 0.9741 - lr: 0.0037
Epoch 5/25
391/391 [==============================] - 2s 4ms/step - loss: 0.0736 - accuracy: 0.9742 - val_loss: 0.0727 - val_accuracy: 0.9748 - lr: 0.0046
Epoch 6/25
391/391 [==============================] - 1s 3ms/step - loss: 0.0712 - accuracy: 0.9742 - val_loss: 0.0712 - val_accuracy: 0.9743 - lr: 0.0055
Epoch 7/25
391/391 [==============================] - 1s 3ms/step - lo

In [14]:
# Evaluate the model using only the activity-related data
evaluation = model.evaluate(
    test_activity,
    test_targets_activity_cat
)

print(evaluation)

335/335 [==============================] - 0s 1ms/step - loss: 0.0701 - accuracy: 0.9755
[0.07013896852731705, 0.9754503965377808]


In [15]:
# Save the model to an H5 file
model.save('domestic.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

In [16]:
# Generate predictions for all inputs
predictions_activity = model.predict([windows_activity])

1116/1116 [==============================] - 2s 1ms/step


In [17]:
import numpy as np
from tensorflow.keras.utils import to_categorical

def compute_anomaly_scores(predictions, actuals):
    # For categorical predictions, convert actuals to one-hot for comparison
    actuals_one_hot = to_categorical(actuals, num_classes=predictions.shape[-1])
    
    max_predictions = np.max(predictions, axis=-1)
    actual_predictions = np.sum(predictions * actuals_one_hot, axis=-1)  # Extract the probability of the actual class
    
    anomaly_scores = (max_predictions - actual_predictions) / max_predictions
    
    return anomaly_scores

# Assuming targets_id, targets_resource, targets_activity, targets_role are the true values for these attributes
anomaly_scores_activity = compute_anomaly_scores(predictions_activity, targets_activity)

In [18]:
import numpy as np

def classify_cases(anomaly_scores_activity,  threshold=0.98):
    # Ensure all inputs are numpy arrays of the same shape
    anomaly_scores_activity = np.array(anomaly_scores_activity).flatten()

    # Find the maximum anomaly score across all attributes for each case
    max_scores = np.maximum.reduce([anomaly_scores_activity])

    # Classify cases as anomalous if the maximum anomaly score exceeds the threshold
    anomalous_cases = max_scores > threshold
    
    return anomalous_cases

# Now use the anomaly scores for resource, activity, and timestamp in the classification
anomalous_cases = classify_cases(anomaly_scores_activity)

# Mapping

### True: anomaly, False: no anomaly

In [19]:
# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted'] = anomalous_cases
mapping

,case,predicted
0,0,False
1,0,False
2,1,False
3,1,False
4,1,False
...,...,...
35705,10498,False
35706,10498,False
35707,10499,False
35708,10499,False


In [20]:
case_prediction = mapping.groupby('case')['predicted'].any()
case_prediction


case
0        False
1        False
2        False
3        False
4        False
         ...  
10495    False
10496    False
10497    False
10498    False
10499    False
Name: predicted, Length: 10500, dtype: bool

# Ground Truth

In [21]:
def generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking):
    from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments
    from pm4py.algo.conformance.alignments.petri_net import variants
    from pm4py.objects.petri_net.utils import align_utils
    max_events=0
    for trace in log:
        counter=0
        for event in trace:
            counter+=1
        if counter > max_events:
            max_events=counter
    parameters={}
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_SYNC_COST_FUNCTION] = list(map(lambda i: .1*i, range(max_events*2)))
    parameters[alignments.Variants.VERSION_STATE_EQUATION_A_STAR.value.Parameters.PARAM_TRACE_COST_FUNCTION]=list(map(lambda i: align_utils.STD_MODEL_LOG_MOVE_COST-.1*i, range(max_events*2)))
    aligned_traces = alignments.apply_log(log, net, initial_marking, final_marking, variant=variants.state_equation_a_star, parameters=parameters)
    return aligned_traces

In [22]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/Model_ DomesticDeclarations.bpmn")

# 3. Convert the BPMN to a Petri net
net, initial_marking, final_marking = pm4py.convert_to_petri_net(bpmn_graph)

aligned_traces = generate_alignments_adjusted_tracecost_pkl(log, net, initial_marking, final_marking)

aligning log, completed variants ::   0%|          | 0/99 [00:00<?, ?it/s]

In [23]:
def extract_conformance_status_by_fitness(aligned_traces):
    conformance_status = []
    for alignment in aligned_traces:
        fitness = alignment['fitness']
        # If the fitness is 1.0, the trace is conforming
        if fitness == 1.0:
            conformance_status.append(0)
        else:
            conformance_status.append(1)
    return conformance_status

# Get the conformance status list from the aligned traces
conformance = extract_conformance_status_by_fitness(aligned_traces)

In [24]:
ground_truth = pd.DataFrame({'conformity': conformance})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth

,conformity,predicted
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
10495,0,0
10496,0,0
10497,0,0
10498,0,0


# Evaluation

In [25]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [26]:
precision_dev = TP / (TP + FP)
print(f"Precision Dev: {precision_dev:.2f}")

Precision Dev: 0.96


In [27]:
recall_dev = TP / (TP + FN)
print(f"Recall Dev: {recall_dev:.2f}")

Recall Dev: 0.02


In [28]:
precision_no_dev = TN / (TN + FN)
print(f"Precision No Dev: {precision_no_dev:.2f}")

Precision No Dev: 0.70


In [29]:
recall_no_dev = TN / (TN + FP)
print(f"Recall No Dev: {recall_no_dev:.2f}")

Recall No Dev: 1.00


In [30]:
from sklearn.metrics import roc_auc_score

auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
print(f"AUC-ROC: {auc_roc:.2f}")

AUC-ROC: 0.51
